### Variables are just Memory References

Objects are stored on the heap at different memory addresses.

e.g. `my_var = 10`

my_var --> Reference (0x100) --> Memory Address (0x100) --> 10

[ ! ] `my_var` just references the memory address which holds the actual value.

We can find the memory address referenced by a variable by using the 
`id()` function, this returns a base-10 number. You can convert this to hexadecimal by using the `hex()` function.

In [1]:
my_var = 10

In [2]:
id(my_var)

139887429536336

In [3]:
hex(id(my_var))

'0x7f3a148bca50'

__Reference Counting__

The Python Memory Manager keeps track of the amount of references to a particular memory address.

We can have 2 references to the same memory address.

e.g.

`my_var = 10`

`other_var = my_var`

my_var ----->

            0x100 --> 10
               
other_var -->

[ ! ] When all references to an address are deleted or out of scope, 
the Python Memory Manager will allow that address to be re-allocated, essentially deleting the object from memory.

__Finding the Reference Count__

[ ! ] This increases the reference count by 1, since variables are passed by reference in Python.

In [4]:
import sys

a = [1, 2, 3]

sys.getrefcount(a)

2

[ ! ] This will avoid the extra reference count

In [5]:
import ctypes

def true_ref_count(address: int):
    return ctypes.c_long.from_address(address).value

In [6]:
b = [2, 4, 6]

true_ref_count(id(b))

1

In [7]:
c = [3, 5, 7]
d = c

true_ref_count(id(c))

2

### Garbage Collection

__Circular References__

my_var --> Object A --> Object B --> Object A

[ ! ] When `my_var` is deleted, Object A still is referenced by Object B, so its space is not re-allocated. This results in a __memory leak__.

The garbage collector can identify these circular references and clean them  up.

__Garbage Collection__

- Can be controlled programmatically using the `gc` module.
- runs periodically on its own
- can be called manually
- by default is turned _on_, but can be turned _off_ for performance reasons
    * __Beware!__ Be sure code does not create circular references, otherwise this will result in memory leaks.


In [8]:
import ctypes
import gc

def true_ref_count(address: int):
    return ctypes.c_long.from_address(address).value

def object_exists(address: int):
    for obj in gc.get_objects():
        if id(obj) == address:
            return True
    
    return False

In [9]:
class A:
    def __init__(self):
        self.b = B(self)
        
        print(f"A - self: {hex(id(self))}\n b: {hex(id(self.b))}")


In [10]:
class B:
    def __init__(self, a):
        self.a = a
        
        print(f"B - self: {hex(id(self))}\n a: {hex(id(self.a))}")

In [11]:
gc.disable()

In [12]:
my_var = A()

B - self: 0x7f3a0c3c4100
 a: 0x7f3a0c3c4340
A - self: 0x7f3a0c3c4340
 b: 0x7f3a0c3c4100


In [13]:
a_id = id(my_var)
b_id = id(my_var.b)

In [14]:
true_ref_count(a_id)

2

In [15]:
true_ref_count(b_id)

1

In [16]:
object_exists(a_id)

True

In [17]:
object_exists(b_id)

True

In [18]:
my_var = None

In [19]:
true_ref_count(a_id)

1

In [28]:
true_ref_count(b_id)

1

__Variable Re-Assignment__

Here, my_var references an object at a certain memory address

In [32]:
my_var = 10

print(hex(id(my_var)))

0x7f3a148bca50


Here, the value at the original memory address does NOT change.

my_var simply references a *different* address with the new value

In [33]:
my_var = 15

print(hex(id(my_var)))

0x7f3a148bcaf0


Even when incrementing a value, the value of the original address
is not modified.

A new address with the incremented value is created and the reference
of my_var is changed

[ ! ] In fact, the value of `int` objects can never be changed (immutability)

In [34]:
my_var = my_var + 5

print(hex(id(my_var)))

0x7f3a148bcb90


Somewhat surprisingly, here we can see that when 2 variables are assigned to the same value, even in separate statements, they share the same reference!

In [37]:
x = 10
y = 10

In [38]:
print(hex(id(x)))
print(hex(id(y)))

0x7f3a148bca50
0x7f3a148bca50
